<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_sac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil

df = "audpred60.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")

restore = True

if restore == True:
  _ = shutil.copy("/content/drive/My Drive/sac_one_step.ckpt.data-00000-of-00001","/content")
  _ = shutil.copy("/content/drive/My Drive/sac_one_step.ckpt.index","/content")
  _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")

output_size = 3
  
# 短いステップサイズを使用することでポジションを長期保有することによって得られるアクション選択を是正し、fxという明確は有限がない状態での総報酬を最大化する
# という目的を制限し、短期報酬を高めることができる。また、長すぎるステップサイズは制御することが困難になる。オーバーフィット対策に頻繁に状態の分布を変えることで
# 対処する。よって、すべての状態で最善の行動を生み出すことができるエージェントを開発する。
  
def actor_work(queues,epsilon, num):
  sess = tf.InteractiveSession()
  actor = Actor(df, 5, num, epsilon, sess,STEP_SIZE=120,OUTPUT_SIZE=output_size, save=True, saver_path="sac_one_step.ckpt", restore=restore)
  actor.run(queues, 10, 1000,200,24,n=4)

def leaner_work(queues):
        sess = tf.InteractiveSession()
        leaner = Leaner(df, 5, sess,output_size,'/gpu:0', save=True, saver_path="sac_one_step.ckpt", restore=restore)
        leaner.leaner(queues,files)

In [0]:
%cd /content

queue = mp.Queue()
ps = [mp.Process(target=leaner_work, args=(queue,))]
for i in range(3):
  epsilon = 0.0 if i == 0 else np.random.rand()
  true = True
  while true:
    if epsilon >= 0.2 or i == 0:
      true = False
    else:
      epsilon = np.random.rand()
  epsilon = 1.0 if i == 1 else epsilon
  ps.append(mp.Process(target=actor_work, args=(queue,epsilon,i)))
            
for p in ps:
    p.start()
    time.sleep(1)

for p in ps:
    p.join()

td error:  21815.838
td error:  12080.44
td error:  16805.582
action probability =  [0.36134453781512604, 0.29411764705882354, 0.3445378151260504]
trade accuracy =  0.5540540540540541
epoch: 1440, total rewards: 1263.000000, mean rewards: -0.382387
td error:  19596.07
td error:  16360.608
td error:  14856.189
td error:  25334.86
td error:  12352.253
action probability =  [0.25210084033613445, 0.35294117647058826, 0.39495798319327724]
trade accuracy =  0.5454545454545454
epoch: 1410, total rewards: 13116.000000, mean rewards: 2.367126
td error:  14466.746
td error:  12054.766
action probability =  [0.23529411764705882, 0.36134453781512604, 0.40336134453781514]
trade accuracy =  0.4925373134328358
epoch: 1440, total rewards: 6589.000000, mean rewards: 0.930555
td error:  8218.002
td error:  12510.576
td error:  4239.136
td error:  11026.79
td error:  4441.6533
td error:  8413.078
td error:  9164.828
td error:  12242.962
td error:  21514.793
td error:  21312.582
td error:  4404.3853
td er

In [0]:
from multiprocessing import Process, Queue

def f(q):
  for i in range(100):
    rand = np.random.RandomState()
    if rand.rand() < 0.2:
      action = rand.choice(3, p=(0.2,0.3,0.5))
      print(i)
      print(action)

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

In [0]:
a = np.asanyarray([[1,1,1,1],[0,0,0,0]])

In [0]:
b = [i[0][0] for i in a]

In [0]:
import urllib.request
import urllib.parse


url = 'https://www.googleapis.com/drive/v2/files/trash'
f = urllib.request.urlopen(url)